In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [9]:
batch_size = 50
total_batch = mnist.train.num_examples // batch_size

x = tf.placeholder(tf.float32,[None, 784])
y = tf.placeholder(tf.float32,[None, 10])

# 改变x的格式，将x改变为图片维度，[batch, in_height, in_width, in_channels]
x_image = tf.reshape(x,[-1, 28, 28, 1])

##########################################
## 第一层卷积层
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1)) # 卷积窗口大小为3 * 3 * 1，从32个卷积核得到取32个特征值
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))                           

h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding="SAME") # 卷积
# 添加激活函数
activation_conv1 = tf.nn.relu(h_conv1 + b_conv1)

#########################################
# 池化层

h_pool1 = tf.nn.max_pool(activation_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

#########################################
### 第二层卷积层
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))

h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding="SAME" )  ## 计算卷积

activation_conv2 = tf.nn.relu(h_conv2 + b_conv2)  ## 使用relu激活函数


########################################
## 池化层
h_pool2 = tf.nn.max_pool(activation_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")


########################################
##  因为卷积的padding方式使用的是SAME padding，所以每次卷积之后图片的大小不变
##  但是池化的步长为2， 所以每次池化之后，图片的大小变为原来的一半
## 28 * 28 的图片第一次卷积之后还是28 * 28
## 第一次池化之后变为 14 * 14
## 第二次卷积之后变为 14 * 14 
## 第二次池化之后图像变为 7* 7
#### 这样经过两次卷积和池化之后，会生成7 * 7 *64 的特征平面
########################################
# 全连接层 1
W_fcn1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1))  # 1024 为定义的神经元个数
b_fcn1 = tf.Variable(tf.constant(0.1, shape=[1024]))

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])  # 将池化后的图片扁平化为1维
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fcn1) + b_fcn1) # 求第一个全连接层的输出


keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob=keep_prob)
#######################################
## 全连接层 2

W_fcn2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fcn2 = tf.Variable(tf.constant(0.1,shape=[10]))

prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fcn2) + b_fcn2)  # 计算输出， 使用softmax函数计算概率值


#########################################
## 代价函数

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction)) # 使用交叉熵代价函数

## 优化函数

optimizer = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

##########################################
# 创建session 窗口
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.7})
        acc = 0
        for i in range(total_batch):
            batch_test_xs,batch_test_ys = mnist.test.next_batch(batch_size)
            acc += sess.run(accuracy, feed_dict={x:batch_test_xs, y: batch_test_ys, keep_prob:1.0})
        print("Iter " + str(epoch) + ", test accuracy = " + str(acc/total_batch))


Iter 0, test accuracy = 0.959745453325
Iter 1, test accuracy = 0.974654547247
Iter 2, test accuracy = 0.982163639773
Iter 3, test accuracy = 0.982745458809
Iter 4, test accuracy = 0.983600003394
Iter 5, test accuracy = 0.988272731575
Iter 6, test accuracy = 0.989454550093
Iter 7, test accuracy = 0.989090913751
Iter 8, test accuracy = 0.989654549685
Iter 9, test accuracy = 0.989018184976
Iter 10, test accuracy = 0.99207273104
Iter 11, test accuracy = 0.990472731374
Iter 12, test accuracy = 0.988454549584
Iter 13, test accuracy = 0.989072731679
Iter 14, test accuracy = 0.991636367061
Iter 15, test accuracy = 0.991090913523
Iter 16, test accuracy = 0.990763640512
Iter 17, test accuracy = 0.993000003479
Iter 18, test accuracy = 0.991654549187
Iter 19, test accuracy = 0.992018186071
Iter 20, test accuracy = 0.991981822252


# refactor code
提取一些公有的函数  
使代码简化

In [ ]:
batch_size = 50
total_batch = mnist.train.num_examples // batch_size

def weight_init(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_init(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")

def max_pool_2d(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")


x = tf.placeholder(tf.float32,[None, 784])
y = tf.placeholder(tf.float32,[None, 10])

# 改变x的格式，将x改变为图片维度，[batch, in_height, in_width, in_channels]
x_image = tf.reshape(x,[-1, 28, 28, 1])

##########################################
## 第一层卷积层
W_conv1 = weight_init([5, 5, 1, 32]) # 卷积窗口大小为3 * 3 * 1，从32个卷积核得到取32个特征值
b_conv1 = bias_init([32])              

h_conv1 = conv2d(x_image, W_conv1) # 卷积

activation_conv1 = tf.nn.relu(h_conv1 + b_conv1) # 添加激活函数

#########################################
# 池化层

h_pool1 = max_pool_2d(activation_conv1)

#########################################
### 第二层卷积层
W_conv2 = weight_init([5, 5, 32, 64])
b_conv2 = bias_init([64])

h_conv2 = conv2d(h_pool1, W_conv2)  ## 计算卷积
activation_conv2 = tf.nn.relu(h_conv2 + b_conv2)  ## 使用relu激活函数

########################################
## 池化层
h_pool2 = max_pool_2d(activation_conv2)

########################################
##  因为卷积的padding方式使用的是SAME padding，所以每次卷积之后图片的大小不变
##  但是池化的步长为2， 所以每次池化之后，图片的大小变为原来的一半
## 28 * 28 的图片第一次卷积之后还是28 * 28
## 第一次池化之后变为 14 * 14
## 第二次卷积之后变为 14 * 14 
## 第二次池化之后图像变为 7* 7
#### 这样经过两次卷积和池化之后，会生成7 * 7 *64 的特征平面
########################################
# 全连接层 1
W_fcn1 = weight_init([7 * 7 * 64, 1024])  # 1024 为定义的神经元个数
b_fcn1 = bias_init([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])  # 将池化后的图片扁平化为1维
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fcn1) + b_fcn1) # 求第一个全连接层的输出


keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob=keep_prob)
#######################################
## 全连接层 2

W_fcn2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fcn2 = tf.Variable(tf.constant(0.1,shape=[10]))

prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fcn2) + b_fcn2)  # 计算输出， 使用softmax函数计算概率值


#########################################
## 代价函数

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction)) # 使用交叉熵代价函数

## 优化函数

optimizer = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

##########################################
# 创建session 窗口
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.7})
        acc = 0
        for i in range(total_batch):
            batch_test_xs,batch_test_ys = mnist.test.next_batch(batch_size)
            acc += sess.run(accuracy, feed_dict={x:batch_test_xs, y: batch_test_ys, keep_prob:1.0})
        print("Iter " + str(epoch) + ", test accuracy = " + str(acc/total_batch))


Iter 0, test accuracy = 0.965690909136
Iter 1, test accuracy = 0.978963639303
Iter 2, test accuracy = 0.982981821678
Iter 3, test accuracy = 0.985236366554
Iter 4, test accuracy = 0.986563640508
Iter 5, test accuracy = 0.98758182168
Iter 6, test accuracy = 0.989181822376
Iter 7, test accuracy = 0.99078182215
Iter 8, test accuracy = 0.990690912984
Iter 9, test accuracy = 0.990472731048
Iter 10, test accuracy = 0.990472731048
Iter 11, test accuracy = 0.990618185834
Iter 12, test accuracy = 0.99132727688
Iter 13, test accuracy = 0.990581821366
Iter 14, test accuracy = 0.991018185995
Iter 15, test accuracy = 0.990854549516
Iter 16, test accuracy = 0.990890913497
Iter 17, test accuracy = 0.990418185863
Iter 18, test accuracy = 0.992036367221
